In [17]:
import json
import os
import sys
sys.path.append("../..")

from evals.spelling_by_grade import create_full_spelling_prompt, prepare_grade_spelling_eval
from evals.plot_utils import basic_bar_graph, create_table, nested_bar_graph
from evals.eval_utils import ModelType
from evals.eval_list import *
from transformer_lens import HookedTransformer

words_by_grade = prepare_grade_spelling_eval("../data/GradeSpellingEval.txt", ' ')
words_by_grade[3][:10]

[('able', 'A B L E'),
 ('above', 'A B O V E'),
 ('afraid', 'A F R A I D'),
 ('afternoon', 'A F T E R N O O N'),
 ('again', 'A G A I N'),
 ('age', 'A G E'),
 ('air', 'A I R'),
 ('airplane', 'A I R P L A N E'),
 ('almost', 'A L M O S T'),
 ('alone', 'A L O N E')]

In [18]:
model = HookedTransformer.from_pretrained('pythia-2.8b')
tokenizer = model.tokenizer

Using pad_token, but it is not set yet.


Loaded pretrained model pythia-2.8b into HookedTransformer


In [ ]:
single_shot_eval = FULL_SPELLING_EVAL.run_eval(model, ModelType.TRANSFORMER_LENS, tokenizer, words_by_grade, 5, should_tqdm=False)

Assessing Grade 1


  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
import copy

adjusted_eval = {'data': {i: [] for i in single_shot_eval['data'].keys()}, 'accuracy': {i: 0 for i in single_shot_eval['data'].keys()}}

# Adjust for Pythia. Not all of this code may be necessary any more.
for grade in single_shot_eval['data'].keys():
    for i in range(len(single_shot_eval['data'][grade])):
        item = copy.deepcopy(single_shot_eval['data'][grade][i])
        item['word'] = item['answer'].replace(' ', '').lower()
        item['answer'] = ' '.join([char.upper() for char in item['word']])
        resp_split = item['response'].replace('<|endoftext|>', '').split('\n\n')
        for rs in resp_split:
            if rs.startswith(item['word']) and ':' in rs:
                item['formatted_response'] = rs.split(':')[1].strip()
        adjusted_eval['data'][grade].append(item)

adjusted_eval['data'][1][0]

In [ ]:
for grade in adjusted_eval['data'].keys():
    data = adjusted_eval['data'][grade]
    adjusted_eval['accuracy'][grade] = sum([1 if data[i]['answer'] == data[i]['formatted_response'] else 0 for i in range(len(data))]) / len(data)
adjusted_eval['accuracy']

In [ ]:
table = create_table(adjusted_eval['data'])
table